# Aprendizado de Máquina

### Este documento apenas tem objetivo para fins exploratórios. Ele não agrega a nada sobre o relatório principal. Nele, testei e explorei possibilidades acerca de utilizar algoritmos envolvendo Gradient Boosting, porém, devido a inconscitência dos dados, isto é, ser um Random Walk extremamente volátil, junto a um desbalanceamento de emissões de casos, tornou-se dificultoso um trabalho para se chegar em um modelo que apresentasse uma acurácia consistente, se baseando no erro percentual absoluto médio simétrico.

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import gzip
import plotly.graph_objects as go

In [2]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import smape_loss

In [3]:
from numpy import ravel
import sklearn.model_selection
from sktime.forecasting.compose import make_reduction
from skopt import gp_minimize
from lightgbm import LGBMRegressor
from statsmodels.tsa.filters.hp_filter import hpfilter
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import ForecastingGridSearchCV

In [4]:
from sktime.forecasting.naive import NaiveForecaster
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

In [6]:
SEED = 4

In [ ]:
url = 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'
r = requests.get(url, allow_redirects=True)
open('data.csv.gz','wb').write(r.content)
gz = gzip.open('data.csv.gz')
df = pd.read_csv(gz)

## Validação de Modelos

In [ ]:
total_de_casos_amazonas = df.query("state == 'AM'").groupby('date').sum()
total_de_casos_amazonas.index = pd.to_datetime(total_de_casos_amazonas.index)
total_de_casos_amazonas.index.freq = 'D'
total_de_casos_amazonas.drop(columns=['epi_week', 'ibgeID', 'cod_RegiaoDeSaude'], inplace=True)
total_de_casos_amazonas = total_de_casos_amazonas.iloc[:,:4]

In [ ]:
def to_zero(x):
    if x < 0:
        return 0
    return x

In [ ]:
for col in total_de_casos_amazonas.columns:
    cycle, trend = hpfilter(total_de_casos_amazonas[col])
    total_de_casos_amazonas[col + '_' + 'cycle'] = cycle.round()
    total_de_casos_amazonas[col + '_' + 'trend'] = trend.round()

In [ ]:
dici = {'Monday': 1,
       'Tuesday': 2,
       'Wednesday': 3,
       'Thursday': 4,
       'Friday': 5,
       'Saturday': 6,
       'Sunday': 7}
total_de_casos_amazonas['dia_da_semana'] = total_de_casos_amazonas.index.day_name()
total_de_casos_amazonas['dia_da_semana'] = total_de_casos_amazonas['dia_da_semana'].apply(lambda x: dici[x])

In [ ]:
total_de_casos_amazonas